In [1]:
import numpy as np
import torch
from torchvision import datasets, transforms
import copy
import collections
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def getData(dist, dataset):
    idxs = []
    counts = [0 for i in range(10)]
    for i in range(len(dataset)):
        if counts[dataset.target[i]]<dist[y[i]]:
            dx.append(x[i])
            dy.append(y[i])
            counts[y[i]] += 1
        
    return np.array(dx), np.array(dy)

In [8]:
# def noniid(dataset, num_users):
#     """
#     Sample non-I.I.D client data from dataset
#     -> Different clients can hold vastly different amounts of data
#     :param dataset:
#     :param num_users:
#     :return:
#     """
#     num_dataset = len(dataset)
#     idx = np.arange(num_dataset) # create a range from 0 -> dataset length
#     dict_users = {i: list() for i in range(num_users)}
    
#     min_num = int(num_dataset*0.1 / num_users)
#     max_num = int(num_dataset*0.9 / num_users)

#     random_num_size = np.random.randint(min_num, max_num+1, size=num_users) # genreate a random size for each user between min/max
#     print(f"Total number of datasets owned by clients : {sum(random_num_size)}")

#     # total dataset should be larger or equal to sum of splitted dataset.
#     assert num_dataset >= sum(random_num_size)

#     # divide and assign
#     for i, rand_num in enumerate(random_num_size):

#         rand_set = set(np.random.choice(idx, rand_num, replace=False))
#         idx = list(set(idx) - rand_set)
#         dict_users[i] = rand_set

#     return dict_users


In [3]:
def noniid(dataset, num_users,num_classes):
    """
    Sample non-I.I.D client data from dataset
    """
    class_size = 6000

    
    dict_users = {i: list() for i in range(10)}
    min_num = int(class_size*0.1 / num_users)
    max_num = int(class_size*0.9 / num_users)
    print(min_num,max_num)

    for i, worker in enumerate(dict_users):
        rand_set = []
        for j in range(num_classes):
            chosen= np.random.choice([min_num,max_num])
            if chosen == min_num:
                smallest = 0
            else:
                smallest = min_num+1
            rand_number = np.random.randint(smallest,chosen)
            # print(rand_number)
            rand_set.append(rand_number)
        print(rand_set)
        dict_users[i] = rand_set

    print(dict_users)
    return dict_users

In [4]:
def train_dg_split(dataset, num_classes, num_users): 
    dg_idx = []
    train_idx = []
    idxs = np.arange(len(dataset))
    labels = np.array(dataset.targets)

    choice_size = int(len(dataset)*0.2/num_users)
    idxs_labels = np.vstack((idxs, labels))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]
    
    idxs = idxs_labels[0]
    labels = idxs_labels[1]
    
    for i in range(num_classes):
        specific_class = np.extract(labels == i, idxs)
        
        dg = np.random.choice(specific_class,choice_size, replace=False)
        print(dg.shape)
        
        train_tmp = set(specific_class)-set(dg)
        
        dg_idx = dg_idx + list(dg)
        
        train_idx = train_idx + list(train_tmp)
    
    return dg_idx, train_idx

In [5]:
def getDist(y,user_num):
    ax = sns.countplot(x=y)
    ax.set(title="Count of data classes for %s" %user_num)
    plt.show()

In [31]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
dev = torch.device(dev)

num_users = 10
num_classes = 10
trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = datasets.CIFAR10('../data/cifar', train=True, download=True, transform=trans_cifar)

dataset_test = datasets.CIFAR10('../data/cifar', train=False, download=True, transform=trans_cifar)
print(len(dataset),len(dataset_test))

Files already downloaded and verified
Files already downloaded and verified
50000 10000


In [7]:
dg = copy.deepcopy(dataset)
dataset_train = copy.deepcopy(dataset)

dg_idx, dataset_train_idx = train_dg_split(dataset, num_classes,num_users)

dg.targets.clear()
dataset_train.targets.clear()

(1000,)
(1000,)
(1000,)
(1000,)
(1000,)
(1000,)
(1000,)
(1000,)
(1000,)
(1000,)


In [8]:
dg.data = dataset.data[dg_idx]
dataset_train.data = dataset.data[dataset_train_idx]
print((len(dataset)))
print (len(dg_idx))
print (len(dataset_train_idx))

50000
10000
40000


In [9]:
for i in list(dg_idx):
    dg.targets.append(dataset[i][1])
for i in list(dataset_train_idx):
    dataset_train.targets.append(dataset[i][1])

In [10]:
dict_users = noniid(dataset_train, num_users,num_classes)

60 540
[93, 267, 13, 128, 24, 4, 55, 108, 50, 58]
[501, 149, 4, 23, 468, 433, 190, 239, 10, 190]
[513, 3, 166, 54, 469, 366, 11, 401, 281, 170]
[449, 15, 38, 52, 488, 512, 382, 6, 27, 282]
[128, 508, 442, 20, 374, 76, 385, 1, 102, 55]
[531, 48, 96, 120, 8, 206, 52, 53, 45, 500]
[123, 54, 331, 194, 182, 44, 20, 368, 139, 272]
[32, 499, 468, 524, 452, 453, 299, 37, 456, 36]
[43, 49, 22, 385, 0, 18, 126, 39, 174, 252]
[18, 46, 114, 371, 434, 281, 407, 311, 14, 53]
{0: [93, 267, 13, 128, 24, 4, 55, 108, 50, 58], 1: [501, 149, 4, 23, 468, 433, 190, 239, 10, 190], 2: [513, 3, 166, 54, 469, 366, 11, 401, 281, 170], 3: [449, 15, 38, 52, 488, 512, 382, 6, 27, 282], 4: [128, 508, 442, 20, 374, 76, 385, 1, 102, 55], 5: [531, 48, 96, 120, 8, 206, 52, 53, 45, 500], 6: [123, 54, 331, 194, 182, 44, 20, 368, 139, 272], 7: [32, 499, 468, 524, 452, 453, 299, 37, 456, 36], 8: [43, 49, 22, 385, 0, 18, 126, 39, 174, 252], 9: [18, 46, 114, 371, 434, 281, 407, 311, 14, 53]}


In [27]:
x = {11,22,55,66,77}
type(x)
y = {111,22,554,66,7337,454}
x -y

{11, 55, 77}

In [30]:
total = 0
target_np = np.asarray(dataset.targets)
num_dataset = len(dataset)
idx = np.arange(num_dataset)
user_database = {i: list() for i in range(10)}
used = []
# for loop to go through all the users
for i in range (len(dict_users)):
    leng = len(dict_users[i])
    total += leng
    print('worker', i, 'data count:',leng)
    print(dict_users[i])
    # for loop to go through all the classes for each user
    for j, class_counter in enumerate(dict_users[i]):
        print(class_counter)
        class_indices = list(np.where(target_np == j)[0]) # dataset labels with current class
        class_indices = list(set(class_indices) - set(used))
        
        class_set = set(np.random.choice(class_indices, class_counter, replace=False))
        used += class_set
        print('used length:',len(used))
        user_database[i] = class_set
        

print(user_database)
    # targets = [dataset_train.targets[i] for i in dict_users[i]]
    # print(targets)
    # counter = collections.Counter(targets)
    # print(counter)

worker 0 data count: 10
[93, 267, 13, 128, 24, 4, 55, 108, 50, 58]
93
used length: 93
267
used length: 360
13
used length: 373
128
used length: 501
24
used length: 525
4
used length: 529
55
used length: 584
108
used length: 692
50
used length: 742
58
used length: 800
worker 1 data count: 10
[501, 149, 4, 23, 468, 433, 190, 239, 10, 190]
501
used length: 1301
149
used length: 1450
4
used length: 1454
23
used length: 1477
468
used length: 1945
433
used length: 2378
190
used length: 2568
239
used length: 2807
10
used length: 2817
190
used length: 3007
worker 2 data count: 10
[513, 3, 166, 54, 469, 366, 11, 401, 281, 170]
513
used length: 3520
3
used length: 3523
166
used length: 3689
54
used length: 3743
469
used length: 4212
366
used length: 4578
11
used length: 4589
401
used length: 4990
281
used length: 5271
170
used length: 5441
worker 3 data count: 10
[449, 15, 38, 52, 488, 512, 382, 6, 27, 282]
449
used length: 5890
15
used length: 5905
38
used length: 5943
52
used length: 5995
488


In [45]:
target_np = np.asarray(dataset.targets)
indices = list(np.where(target_np == 6)[0]) # if you want to keep images with the label 5
print(indices)
# print(dataset.targets[indices])

[0, 19, 22, 23, 25, 72, 95, 103, 104, 117, 124, 125, 132, 143, 151, 154, 164, 187, 200, 204, 209, 210, 224, 228, 231, 232, 234, 235, 242, 243, 245, 248, 249, 286, 292, 298, 313, 326, 327, 347, 350, 351, 355, 361, 368, 387, 409, 437, 451, 452, 464, 473, 488, 525, 529, 532, 552, 556, 571, 587, 588, 591, 619, 620, 625, 633, 640, 645, 651, 655, 667, 680, 682, 692, 710, 718, 720, 721, 728, 738, 745, 755, 770, 781, 807, 818, 819, 836, 837, 838, 854, 862, 863, 896, 899, 903, 914, 920, 921, 923, 929, 931, 935, 937, 960, 961, 985, 1013, 1016, 1017, 1023, 1027, 1031, 1035, 1038, 1047, 1051, 1053, 1063, 1085, 1101, 1126, 1131, 1132, 1154, 1155, 1159, 1161, 1175, 1176, 1191, 1192, 1194, 1210, 1220, 1222, 1228, 1237, 1246, 1248, 1259, 1317, 1318, 1327, 1342, 1360, 1362, 1367, 1371, 1374, 1393, 1402, 1403, 1420, 1436, 1447, 1453, 1484, 1485, 1529, 1531, 1536, 1537, 1550, 1552, 1563, 1579, 1584, 1585, 1596, 1613, 1627, 1628, 1635, 1637, 1642, 1646, 1678, 1687, 1690, 1722, 1730, 1733, 1741, 1743, 1760

In [5]:
print(type(dict_users[0]))
targets = [dataset_train.targets[i] for i in dict_users[0]]
print(targets)

NameError: name 'dict_users' is not defined